In [1]:
import argparse

import torch
import torch.nn as nn
import numpy as np
import pandas as pd

## Two separate layers of RNN

In [2]:
# embedding_dim = 128
# class Encoder2Layer(nn.Module):
#     def __init__(self, seq_len, n_features, n_layers):
#         super(Encoder2Layer, self).__init__()
#         self.seq_len = seq_len
#         self.n_features = n_features
#         self.n_layers = n_layers
#         self.hidden_dim = 2 * embedding_dim
#         self.rnn1 = nn.LSTM(
#             input_size=self.n_features,
#             hidden_size=self.hidden_dim,
#             num_layers=self.n_layers,
#             batch_first=True
#         )
#         self.rnn2 = nn.LSTM(
#             input_size=self.hidden_dim,
#             hidden_size=embedding_dim,
#             num_layers=self.n_layers,
#             batch_first=True
#         )
#     def forward(self, x):
#         # x: (4, 16, 64)
#         batch_size = x.shape[0]
#         x, _ = self.rnn1(x) # (4, 16, 128)
#         print(f'After first RNN, output shape: {x.shape}')
#         x, (hidden, _) = self.rnn2(x)
#         # (batch_size, seq_len, embedding_dim)
#         print(f'After second RNN, output shape: {x.shape}') # (4, 16, 128)
#         # (n_layers, batch_size, embedding_dim)
#         print(f'After second RNN, hidden state shape: {hidden.shape}') # (1, 4, 128)
#         return hidden.reshape(batch_size, -1)


In [3]:
# x = torch.rand(4, 16, 64)
# encoder = Encoder2Layer(seq_len=16, n_features=64, n_layers=1)
# out = encoder(x)
# out.shape

In [4]:
# class Decoder2Layer(nn.Module):
#     def __init__(self, seq_len, n_features, n_layers):
#         super(Decoder2Layer, self).__init__()
#         self.seq_len = seq_len
#         self.n_features = n_features
#         self.n_layers = n_layers
#         self.hidden_dim = 2 * embedding_dim
#         self.rnn1 = nn.LSTM(
#             input_size=embedding_dim,
#             hidden_size=embedding_dim,
#             num_layers=self.n_layers,
#             batch_first=True
#         )
#         self.rnn2 = nn.LSTM(
#             input_size=embedding_dim,
#             hidden_size=self.hidden_dim,
#             num_layers=self.n_layers,
#             batch_first=True
#         )
#         self.fc = nn.Linear(self.hidden_dim, self.n_features)
#     def forward(self, x):
#         # x: (batch_size, embedding_dim)
#         x = x.repeat(self.seq_len, 1)
#         print(f'After repeat, x shape: {x.shape}')
#         x = x.reshape(4, self.seq_len, embedding_dim)
#         print(f'After reshaping, x shape: {x.shape}')
#         x, (hidden, cell) = self.rnn1(x)
#         print(f'After first RNN, output shape: {x.shape}') # (4, 16, 128)
#         print(f'After first RNN, hidden state shape: {hidden.shape}')
#         x, (hidden, cell) = self.rnn2(x)
#         print(f'After second RNN, output shape: {x.shape}') # (4, 16, 128)
#         print(f'After second RNN, hidden state shape: {hidden.shape}')
#         x = torch.squeeze(x)
#         x = self.fc(x)
#         return x

In [5]:
# decoder = Decoder2Layer(seq_len=16, n_features=64, n_layers=1)
# dec_out = decoder(out)
# dec_out.shape

## Single layer of RNN

In [6]:
# embedding_dim = 128
# class Encoder(nn.Module):
#     def __init__(self, seq_len, n_features, n_layers):
#         super(Encoder, self).__init__()
#         self.seq_len = seq_len
#         self.n_features = n_features
#         self.n_layers = n_layers
#         self.hidden_dim = 4*embedding_dim
#         self.rnn = nn.LSTM(
#             input_size=self.n_features,
#             hidden_size=self.hidden_dim,
#             num_layers=self.n_layers,
#             batch_first=True
#         )
#     def forward(self, x):
#         # x: (4, 16, 64)
#         batch_size = x.shape[0]
#         _, (hidden, _) = self.rnn(x)
#         # (num_layers, batch_size, hidden_dim)
#         print(f'After RNN, hidden size: {hidden.shape}')    # (2, batch_size, hidden_size)

#         return hidden.reshape(batch_size, -1)

In [7]:
# x = torch.rand(4, 16, 64)
# encoder = Encoder(seq_len=16, n_features=64, n_layers=1)
# out = encoder(x)
# out.shape

In [8]:
# class Decoder(nn.Module):
#     def __init__(self, seq_len, n_features, n_layers):
#         super(Decoder, self).__init__()
#         self.seq_len = seq_len
#         self.n_features = n_features
#         self.n_layers = n_layers
#         self.hidden_dim = 4*embedding_dim
#         self.rnn = nn.LSTM(
#             input_size=self.n_layers*self.hidden_dim,
#             hidden_size=embedding_dim,
#             num_layers=self.n_layers,
#             batch_first=True
#         )
#         self.fc = nn.Linear(embedding_dim, self.n_features)
    
#     def forward(self, x):
#         # x: (batch_size, embedding_dim)
#         x = x.repeat(1, self.seq_len, 1)
#         print(f'After repeat, x shape: {x.shape}')
#         x = x.reshape(4, self.seq_len, self.n_layers*self.hidden_dim)
#         print(f'After reshaping, x shape: {x.shape}')
#         x, (hidden, cell) = self.rnn(x)
#         print(f'After first RNN, output shape: {x.shape}') # (4, 16, 128)
#         print(f'After first RNN, hidden state shape: {hidden.shape}')
#         x = torch.squeeze(x)
#         x = self.fc(x)
#         return x        

In [9]:
# decoder = Decoder(seq_len=16, n_features=64, n_layers=1)
# dec_out = decoder(out)
# dec_out.shape

In [10]:
class Encoder(nn.Module):
    
    def __init__(
        self, args, seq_len, n_features, n_layers, dropout, batch_size
    ):
        super(Encoder, self).__init__()
        self.args = args
        self.seq_len = seq_len
        self.n_features = n_features
        self.hidden_dim = self.args.hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.batch_size = batch_size
        self.rnn = nn.LSTM(
            input_size=self.n_features,
            hidden_size=self.hidden_dim,
            batch_first=True,
            num_layers=self.n_layers,
            dropout=self.dropout,
        )

    def forward(self, x):
        batch_size = self.batch_size
        _, (hidden, _) = self.rnn(x)
        # hidden size: (num_layers, batch_size, hidden_size)
        # hidden = hidden.reshape(
        #     batch_size, -1
        # )  # (batch_size, num_layers*hidden_size)
        hidden = hidden.transpose(0, 1).contiguous().view(
            -1, self.n_layers*self.hidden_dim
        )
        return hidden


class Decoder(nn.Module):
    def __init__(
        self, args, seq_len, n_features, n_layers, dropout, batch_size
    ):
        super(Decoder, self).__init__()
        self.args = args
        self.seq_len = seq_len
        self.n_features = n_features
        self.hidden_dim = self.args.hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.batch_size = batch_size
        self.rnn = nn.LSTM(
            input_size=self.n_layers * self.hidden_dim,
            hidden_size=self.hidden_dim,
            batch_first=True,
            num_layers=self.n_layers,
            dropout=self.dropout,
        )
        self.fc = nn.Linear(self.hidden_dim, self.n_features)

    def forward(self, x):
        # x: (batch_size, num_layers*hidden_size)
        x = x.repeat(1, self.seq_len, 1)
        x = x.reshape(
            -1, self.seq_len, self.n_layers * self.hidden_dim
        )
        x, _ = self.rnn(x)  # x: (batch_size, seq_len, hidden_dim)
        x = self.fc(x)  # x: (batch_size, seq_len, n_features)

        return x


class LSTMAutoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(LSTMAutoencoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions for both encoder and decoder must be equal"

        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder should have same number of layers"

    def forward(self, input):
        # input = torch.unsqueeze(input, 0)
        # encode
        hidden = self.encoder(input)
        # decode
        y = self.decoder(hidden)

        return y.squeeze(0)

In [11]:
X_valid = np.load('X_valid.npy')
y_valid = np.load('y_valid.npy')
X_valid = torch.as_tensor(X_valid, dtype=torch.float32)
y_valid = torch.as_tensor(y_valid, dtype=torch.long)
test_dataset = torch.utils.data.TensorDataset(X_valid, y_valid)
print(test_dataset.__getitem__(0)[0].shape)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)

torch.Size([16, 64])


In [12]:
parser = argparse.ArgumentParser()
args = parser.parse_args('')
args.hidden_size = 32
args.device = 'cuda'

In [13]:
seq_len = 16
n_features = 64
n_layers = 2
pct = 0.3
encoder = Encoder(
    args,
    seq_len=seq_len,
    n_features=n_features,
    n_layers=n_layers,
    dropout=pct,
    batch_size=1
)
decoder = Decoder(
    args,
    seq_len=seq_len,
    n_features=n_features,
    n_layers=n_layers,
    dropout=pct,
    batch_size=1
)
model = LSTMAutoencoder(encoder, decoder)
model = model.to(args.device)
model

LSTMAutoencoder(
  (encoder): Encoder(
    (rnn): LSTM(64, 32, num_layers=2, batch_first=True, dropout=0.3)
  )
  (decoder): Decoder(
    (rnn): LSTM(64, 32, num_layers=2, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=32, out_features=64, bias=True)
  )
)

In [14]:
model = torch.load('lstm_ae.pth')
model = model.to(args.device)

In [15]:
model

LSTMAutoencoder(
  (encoder): Encoder(
    (rnn): LSTM(64, 32, num_layers=2, batch_first=True, dropout=0.3)
  )
  (decoder): Decoder(
    (rnn): LSTM(64, 32, num_layers=2, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=32, out_features=64, bias=True)
  )
)

In [16]:
model.eval()
seq, _ = next(iter(test_loader))
# seq = torch.unsqueeze(seq, 0)
print(seq.shape)
seq = seq.to(args.device)
seq_pred = model(seq)
print(seq_pred.shape)

torch.Size([1, 16, 64])
torch.Size([16, 64])


In [17]:
with torch.no_grad():
    mae = []
    for data in test_loader:
        seq = data[0]
        seq = seq.to(args.device)
        pred_seq = model(seq)
        loss = torch.sum(torch.abs(torch.squeeze(seq, 0) - pred_seq))
        mae.append(loss.cpu().numpy())
error_df = pd.DataFrame(
    {"Reconstruction_error": mae, "Ground_truth": y_valid.tolist()}
)
print(error_df)

     Reconstruction_error  Ground_truth
0                22.28201             0
1               22.173965             0
2               22.230276             0
3               22.294802             0
4               22.253998             0
...                   ...           ...
6728            16.939283             0
6729            17.069946             0
6730            17.401886             0
6731             17.23662             0
6732            17.157166             0

[6733 rows x 2 columns]


In [18]:
error_df[error_df['Ground_truth'] == 1]

,Reconstruction_error,Ground_truth
1547,30.741676,1
1548,30.697445,1
1549,30.76849,1
1550,30.574942,1
1551,30.21465,1
...,...,...
5901,151.29599,1
5902,149.03806,1
5903,146.76474,1
5904,145.10883,1
